In [ ]:
import pandas as pd
import numpy as np
from sklearn.svm import SVR
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.multioutput import MultiOutputRegressor
from sklearn.metrics import mean_squared_error, r2_score
from scipy.stats import loguniform
import matplotlib.pyplot as plt
from preprocessing import *

In [ ]:
X_file = "data/X_matrix.csv"
Y_file = "data/Y_matrix.csv"

print("Chargement des données...")
X = pd.read_csv(X_file)
Y = pd.read_csv(Y_file)

In [ ]:
# Préparer les données
# Vérifier et supprimer les colonnes inutiles comme 'Yeast_ID'
if "Yeast_ID" in X.columns:
    X = X.drop(columns=["Yeast_ID"])
if "Yeast_ID" in Y.columns:
    Y = Y.drop(columns=["Yeast_ID"])

X, Y = preprocessed_data(X, Y)

In [ ]:
print(f"Dimensions après prétraitement : X={X_pre.shape}, Y={Y_pre.shape}")

# Aperçu des données après prétraitement
print("Aperçu des premières lignes des données après prétraitement :")
print(X_pre.head())
print(Y_pre.head())

In [ ]:
# Diviser les données en ensembles d'entraînement et de test
print("Division des données en ensembles d'entraînement et de test...")
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.25, random_state=42)

# Affichage des dimensions des ensembles divisés
print(f"Dimensions de X_train : {X_train.shape}, X_test : {X_test.shape}")
print(f"Dimensions de y_train : {y_train.shape}, y_test : {y_test.shape}")

In [ ]:
# Définir les distributions pour RandomizedSearchCV
param_distributions = {
    "C": loguniform(1e-3, 1e3),
    "epsilon": loguniform(1e-3, 1),
    "gamma": ["scale", "auto"],
    "kernel": ["linear", "rbf"]
}

In [ ]:
# Configurer et entraîner le modèle multitâche
print("Optimisation des hyperparamètres avec RandomizedSearchCV...")
svr = SVR()
multi_svr = MultiOutputRegressor(RandomizedSearchCV(
    estimator=svr,
    param_distributions=param_distributions,
    n_iter=50,  # Nombre d'itérations pour RandomizedSearchCV
    cv=3,
    verbose=1,
    random_state=42,
    n_jobs=-1
))

multi_svr.fit(X_train, y_train)

In [ ]:
# Prédictions
print("Prédictions sur l'ensemble de test...")
y_pred = multi_svr.predict(X_test)

In [ ]:
# Sauvegarde des résultats
y_test_pred_df = pd.DataFrame(y_pred, index=y_test.index, columns=Y.columns)
y_test_pred_df.to_csv('results/y_test_predicted_multitarget_GBM.csv')

In [ ]:
# Calcul des métriques
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
print(f"Mean Squared Error (MSE): {mse}")
print(f"R² Score: {r2}")

In [ ]:
# Afficher les hyperparamètres optimaux
print("Hyperparamètres optimaux pour chaque tâche :")
for i, estimator in enumerate(multi_svr.estimators_):
    print(f"Tâche {i + 1}: {estimator.best_params_}")


In [ ]:
for i in range(y_test.shape[1]):
    plt.figure(figsize=(8, 6))
    plt.scatter(y_test.iloc[:, i], y_pred[:, i], alpha=0.7)
    plt.plot([y_test.iloc[:, i].min(), y_test.iloc[:, i].max()],
             [y_test.iloc[:, i].min(), y_test.iloc[:, i].max()],
             "r--", lw=2)
    plt.title(f"Résultats de la tâche {i + 1}")
    plt.xlabel("Valeurs réelles")
    plt.ylabel("Valeurs prédites")
    plt.show()